Neste projeto, faço o processamento de um conjunto de dados para que seja utilizado por um banco comercial para guiar o processo de aprovação/rejeição na concessão de cartões de crédito. Após o processamento, os dados serão utilizados para a criaçao de um modelo de machine learning para prever a aprovação na obtenção do cartão.

Os dados utilizados neste projeto foram obtidos através do repositório UCI Machine Learning


In [2]:
#Importando o Pandas
import pandas as pd

#Carregando o dataset
cc_approv = pd.read_csv("/Users/computer/Documents/Coding/Projetos/credit approvals data/CreditApproval.csv")

#Conferindo os dados carregados
cc_approv.head(15)

,Gender,Age,Debt,Married,BankCustomer,EducationLevel,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,ApprovalStatus
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43,560,+
2,a,24.5,0.500,u,g,q,h,1.50,t,f,0,f,g,280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120,0,+


# Inspecionando os dados

Podemos ver que o dataset possui uma mistura de dados numéricos e não-numéricos.
Vamos explorá-lo um pouco mais para entender a disposição de seus dados e detectar possíveis problemas que devam ser corrigidos.

In [3]:
# Estatísticas descritivas
cc_approv_description = cc_approv.describe()
print(cc_approv_description)

print('\n')

# Print DataFrame information
cc_approv_info = cc_approv.info()
print(cc_approv_info)




             Debt  YearsEmployed  CreditScore         Income
count  690.000000     690.000000    690.00000     690.000000
mean     4.758725       2.223406      2.40000    1017.385507
std      4.978163       3.346513      4.86294    5210.102598
min      0.000000       0.000000      0.00000       0.000000
25%      1.000000       0.165000      0.00000       0.000000
50%      2.750000       1.000000      0.00000       5.000000
75%      7.207500       2.625000      3.00000     395.500000
max     28.000000      28.500000     67.00000  100000.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Gender          690 non-null    object 
 1   Age             690 non-null    object 
 2   Debt            690 non-null    float64
 3   Married         690 non-null    object 
 4   BankCustomer    690 non-null    object 
 5   EducationLevel  690 non-

Podemos perceber que este data set:
-Contém 4 colunas da dados numéricos (Debt, YearsEmployed, CreditScore e Income) enquanto as outras colunas contém dados do tipo não-numérico.

-Que os dados numéricos variam bastante em termos de amplitude. Enquanto a coluna Debt varia de 0-28, a coluna Income varia de 0-100000.

-Temos alguns valores faltantes que, neste dataset, foram substituídos por "?".

Além disso, uma primeira análise deste dataset indica que colunas como o tipo da habilitação (DriversLicense) e o código postal (ZipCode) não são tão relevantes quanto as outras para prever a concessão de cartão de crédito (análises mais robustas poderiam ser realizadas para avaliar a relevância destes dados, mas estas técnicas estão fora do escopo deste projeto). Desta forma, estas colunas serão excluídas do dataset.

In [5]:
#Excluindo as colunas DriversLicense e ZipCode
cc_approv = cc_approv.drop(["DriversLicense" , "ZipCode"], axis=1)

# Dividindo o dataset entre treino e teste

Os manuais de machine learning preconizam que nenhuma informação dos dados de teste deve ser utilizada para pré-processar os dados de treino. Desta forma, realizei a divisão dos dados entre treino e teste previamente ao processamento destes dados.
Para isso, será utilizada a função train_test_split do sklearn.


In [6]:
# Importando a função train_test_split
from sklearn.model_selection import train_test_split

#Dividindo os dados entre treino e teste
cc_approv_treino, cc_approv_teste = train_test_split(cc_approv, test_size=0.33, random_state=1)

# Lidando com valores faltantes

Vamos substituir os valores faltantes (neste caso marcados como "?") pela expressão "NaN".

In [7]:
import numpy as np

#Substituir os "?" por "NaN" nos dataset treino e teste

cc_approv_treino = cc_approv_treino.replace("?", np.NaN)
cc_approv_teste = cc_approv_teste.replace("?", np.NaN)

O próximo passo foi imputar os valores numéricos faltantes. Neste caso, decidiu-se substituí-los pelo valor médio de cada coluna. 

In [8]:
# Imputar a média nos campos com valores faltantes
cc_approv_treino.fillna(cc_approv_treino.mean(), inplace=True)
cc_approv_teste.fillna(cc_approv_treino.mean(), inplace=True)

# Verificar o número de NaNs em cada coluna. Poderemos observar um total de 0 valores faltantes nas colunas com dados numéricos
print(cc_approv_treino.isnull().sum())
print(cc_approv_teste.isnull().sum())

Gender            6
Age               6
Debt              0
Married           5
BankCustomer      5
EducationLevel    6
Ethnicity         6
YearsEmployed     0
PriorDefault      0
Employed          0
CreditScore       0
Citizen           0
Income            0
ApprovalStatus    0
dtype: int64
Gender            6
Age               6
Debt              0
Married           1
BankCustomer      1
EducationLevel    3
Ethnicity         3
YearsEmployed     0
PriorDefault      0
Employed          0
CreditScore       0
Citizen           0
Income            0
ApprovalStatus    0
dtype: int64


/var/folders/t2/y5xv3kvj3bq_p9_lkwdl_mj00000gn/T/ipykernel_22829/2599794956.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  cc_approv_treino.fillna(cc_approv_treino.mean(), inplace=True)
/var/folders/t2/y5xv3kvj3bq_p9_lkwdl_mj00000gn/T/ipykernel_22829/2599794956.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  cc_approv_teste.fillna(cc_approv_treino.mean(), inplace=True)


Agora vamos processar os valores faltantes nas colunas não-numéricas.
A estratégia escolhida neste projeto foi substituir os falores faltantes pelo valor mais frequente em cada coluna. Contudo, vale salientar que esta estratégia pode variar dependendo de cada projeto, e que a sua escolha deve ser cuidadosa e, se possível, discutida com o time de dados da empresa.

In [9]:
# Iterando em cada coluna de cc_approv_treino
for col in cc_approv_treino.columns:
    # Confere se a coluna é do tipo "Object"
    if cc_approv_treino[col].dtype == 'object':
        # Imputa o valor mais frequente da coluna
        cc_approv_treino = cc_approv_treino.fillna(cc_approv_treino[col].value_counts().index[0])
        cc_approv_teste = cc_approv_teste.fillna(cc_approv_treino[col].value_counts().index[0])
'''Em resumo, o código percorre cada coluna do dataframe "cc_approv_treino" e verifica se ela é do tipo 'object'. Caso True, os 
valores ausentes (NaN) são preenchidos com o valor mais frequente encontrado na respectiva coluna. index[0] retorna o primeiro
índice da série, que corresponde ao valor mais frequente'''

# Verificar o número de NaNs em cada coluna. Poderemos observar que não existem mais valores faltantes nos datasets.
print(cc_approv_treino.isnull().sum())
print(cc_approv_teste.isnull().sum())

Gender            0
Age               0
Debt              0
Married           0
BankCustomer      0
EducationLevel    0
Ethnicity         0
YearsEmployed     0
PriorDefault      0
Employed          0
CreditScore       0
Citizen           0
Income            0
ApprovalStatus    0
dtype: int64
Gender            0
Age               0
Debt              0
Married           0
BankCustomer      0
EducationLevel    0
Ethnicity         0
YearsEmployed     0
PriorDefault      0
Employed          0
CreditScore       0
Citizen           0
Income            0
ApprovalStatus    0
dtype: int64


Sucesso!

Agora, os próximos passos garantirão que este dataset esteja apropriado para o uso em modelos de ML:

-Converter valores não-numéricos para numéricos

-Alterar a escala dos valores numéricos para que eles fiquem mais uniformes.

In [10]:
# Convertendo os valores não-numéricos para numéricos utilizando o método get_dummies
cc_approv_treino = pd.get_dummies(cc_approv_treino)
cc_approv_teste = pd.get_dummies(cc_approv_teste)


# Utilizando a função Reindex para garantir que as colunas do dataset de teste estejam alinhadas com as colunas do dataset de treino
cc_approv_teste = cc_approv_teste.reindex(columns=cc_approv_treino.columns, fill_value=0)

In [13]:
#Utilizando a função MinMaxScaler para alterar a escala dos dados
from sklearn.preprocessing import MinMaxScaler

# Separando "features" e "label" em varáveis separadas. Neste datase o "label" é a última coluna da tabela que diz se o cartão de crédito foi aprovado ou não.
x_treino, y_treino = cc_approv_treino.iloc[:, :-1].values, cc_approv_treino.iloc[:,[-1]].values #separando a coluna label das demais
x_teste, y_teste = cc_approv_teste.iloc[:, :-1].values, cc_approv_teste.iloc[:,[-1]].values #separando a coluna label das demais

# Usando o método MinMaxScaler para mudar a escala dos dados de X_treino and X_teste
escala = MinMaxScaler(feature_range=(0,1))
reescalax_treino = escala.fit_transform(x_treino)
reescalax_teste = escala.transform(x_teste)

In [16]:
cc_approv_treino.head()

,Debt,YearsEmployed,CreditScore,Income,Gender_a,Gender_b,Age_15.17,Age_15.75,Age_15.83,Age_15.92,...,Ethnicity_z,PriorDefault_f,PriorDefault_t,Employed_f,Employed_t,Citizen_g,Citizen_p,Citizen_s,ApprovalStatus_+,ApprovalStatus_-
589,0.580,0.290,7,5124,0,1,0,0,0,0,...,0,0,1,0,1,1,0,0,1,0
570,2.750,1.750,5,58,0,1,0,0,0,0,...,0,0,1,0,1,1,0,0,1,0
361,2.500,0.085,0,4208,0,1,0,0,0,0,...,0,1,0,1,0,1,0,0,0,1
38,0.585,0.250,2,500,0,1,0,0,0,0,...,0,0,1,0,1,1,0,0,1,0
438,1.250,0.000,1,300,1,0,0,0,0,0,...,0,1,0,0,1,1,0,0,0,1


In [19]:
cc_approv_treino.shape

(462, 339)

In [20]:
cc_approv_teste.head()

,Debt,YearsEmployed,CreditScore,Income,Gender_a,Gender_b,Age_15.17,Age_15.75,Age_15.83,Age_15.92,...,Ethnicity_z,PriorDefault_f,PriorDefault_t,Employed_f,Employed_t,Citizen_g,Citizen_p,Citizen_s,ApprovalStatus_+,ApprovalStatus_-
666,11.75,0.250,0,0,1,0,0,0,0,0,...,0,1,0,1,0,1,0,0,0,1
520,7.50,1.500,1,234,0,1,0,0,0,0,...,0,0,1,0,1,1,0,0,1,0
681,1.00,3.000,0,537,0,1,0,0,0,0,...,0,1,0,1,0,1,0,0,0,1
23,14.50,3.085,1,11,1,0,0,0,0,0,...,0,0,1,0,1,1,0,0,1,0
65,1.54,1.540,1,50000,0,1,0,0,0,0,...,0,0,1,0,1,1,0,0,1,0


In [21]:
cc_approv_teste.shape

(228, 339)

In [23]:
cc_approv_treino.to_csv("cc_approv_treino.csv")

In [24]:
cc_approv_teste.to_csv("cc_approv_test.csv")